In [10]:
from langchain_community.document_loaders import TextLoader

In [11]:
loader = TextLoader("greet.txt")
text_documents=loader.load()
text_documents

[Document(metadata={'source': 'greet.txt'}, page_content='hellloooo \n\nhow are you')]

In [12]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")


In [13]:
from langchain_community.document_loaders import WebBaseLoader
import bs4


loader=WebBaseLoader(web_path="https://medium.com/data-science/nlp-101-word2vec-skip-gram-and-cbow-93512ee24314",
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                        class_=("du bh iz ja jb jc")
                     )),)

web_doc=loader.load()
web_doc

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://medium.com/data-science/nlp-101-word2vec-skip-gram-and-cbow-93512ee24314'}, page_content='NLP 101: Word2Vec — Skip-gram and CBOWA crash course in word embedding.Ria Kulshrestha·FollowPublished inTDS Archive·7 min read·Nov 24, 2019--9ListenShareWhat does word embedding mean?Word embedding is just a fancy way of saying numerical representation of words. A good analogy would be how we use the RGB representation for colors.Why do we need word embedding?As a human, intuitively speaking, it doesn’t make much sense in wanting to represent words or any other object in the universe using numbers because numbers are used for quantification and why would one need to quantify words?When in science, we say speed of my car is 45 km/hr we gain a sense of how fast/slow we are driving. If we say my friend is driving at 60 km/hr, we can compare which one of us is going faster. Furthermore, we can calculate where we will be at a certain point in time, when we will r

In [14]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("https://arxiv.org/pdf/2305.14062.pdf")
pdf_doc=loader.load()
pdf_doc

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-01-17T04:11:30+00:00', 'author': 'Yuyang Miao; Harry J. Davies; Danilo P. Mandic;', 'keywords': '', 'moddate': '2024-01-17T04:11:30+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': 'VGTL-net: Amplitude-Independent Machine Learning for PPG through Visibility Graphs and Transfer Learning', 'trapped': '/False', 'source': 'https://arxiv.org/pdf/2305.14062.pdf', 'total_pages': 12, 'page': 0, 'page_label': '1'}, page_content='VGTL-net: Amplitude-Independent Machine Learning for PPG through Visibility Graphs\nand Transfer Learning\nYuyang Miaoa,∗, Harry J. Daviesa, Danilo P. Mandica\na Department of Electrical and Electronic Engineering, Imperial College London, Exhibition Rd, South Kensington, London, SW7 2BX, United Kingdom\nAbstract\nPhotoplethysmography (PPG) refers to the measurement of variat

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
texts = text_splitter.split_documents(pdf_doc)
texts[:2]

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-01-17T04:11:30+00:00', 'author': 'Yuyang Miao; Harry J. Davies; Danilo P. Mandic;', 'keywords': '', 'moddate': '2024-01-17T04:11:30+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': 'VGTL-net: Amplitude-Independent Machine Learning for PPG through Visibility Graphs and Transfer Learning', 'trapped': '/False', 'source': 'https://arxiv.org/pdf/2305.14062.pdf', 'total_pages': 12, 'page': 0, 'page_label': '1'}, page_content='VGTL-net: Amplitude-Independent Machine Learning for PPG through Visibility Graphs\nand Transfer Learning\nYuyang Miaoa,∗, Harry J. Daviesa, Danilo P. Mandica\na Department of Electrical and Electronic Engineering, Imperial College London, Exhibition Rd, South Kensington, London, SW7 2BX, United Kingdom\nAbstract\nPhotoplethysmography (PPG) refers to the measurement of variat

In [16]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

db = Chroma.from_documents(
    texts[:6],
    OllamaEmbeddings(model="llama3.2:1b"),
)


/tmp/ipykernel_630659/2955505664.py:6: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  OllamaEmbeddings(model="llama3.2:1b"),


In [17]:
query="An additional easily observable feature of PPG is the heart rate (HR)what is Signal To Graph: Visibility Graph"
res=db.similarity_search(query)
res[0].page_content

'VGTL-net: Amplitude-Independent Machine Learning for PPG through Visibility Graphs\nand Transfer Learning\nYuyang Miaoa,∗, Harry J. Daviesa, Danilo P. Mandica\na Department of Electrical and Electronic Engineering, Imperial College London, Exhibition Rd, South Kensington, London, SW7 2BX, United Kingdom\nAbstract\nPhotoplethysmography (PPG) refers to the measurement of variations in blood volume using light and is a feature of most wearable\ndevices. The PPG signals provide insight into the body’s circulatory system and can be employed to extract various bio-features,\nsuch as heart rate and vascular ageing. Although several algorithms have been proposed for this purpose, many exhibit limitations,\nincluding heavy reliance on human calibration, high signal quality requirements, and a lack of generalisation. In this paper, we\nintroduce a PPG signal processing framework that integrates graph theory and computer vision algorithms, to provide an analysis'

In [18]:
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
    Answer the question based on the context provided. If the answer is not in the context, say "I don't know
    think step by step before answering provide detail answer".
    <context>
    {context}
    </context>
    Question: {input}
"""
)


In [19]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

# Define the prompt template with 'context' and 'input' variables
prompt = ChatPromptTemplate.from_template(
    """
    Answer the question based on the context provided. If the answer is not in the context, say "I don't know."
    Think step by step before answering and provide a detailed answer.
    <context>
    {context}
    </context>
    Question: {input}
    """
)

# Initialize the Ollama language model
llm = Ollama(model="llama3.2:1b")

# Create the document chain, specifying 'context' as the document variable name
doc_chain = create_stuff_documents_chain(
    llm,
    prompt,
    document_variable_name="context"
)

retriever=db.as_retriever()
retriever


retrieval_chain=create_retrieval_chain(retriever,doc_chain)

res=retrieval_chain.invoke(
    {
        "input": "what is Signal To Graph: Visibility Graph"
    }
)
res["answer"]

'Based on the context provided, I don\'t know. \n\nThe author of the paper mentions that they introduce a PPG signal processing framework called "Signal To Graph: Visibility Graph" in their introduction. However, they do not explicitly define or explain what "Visibility Graph" is within this specific framework.\n\nTo provide a detailed answer, I would need to look beyond the provided context and research papers. \n\nFrom my understanding of graph theory and computer vision algorithms, a Visibility Graph (VG) refers to a data structure used in various applications, including image processing, object recognition, and machine learning. It is a way of representing complex data as a graph where vertices represent data points or objects, and edges connect related or adjacent elements.\n\nIn the context of signal processing, particularly PPG signals, a Visibility Graph could potentially be used to analyze and process the signal in a more efficient and effective manner. However, without furthe